In [ ]:
import tensorflow as tf
from keras.layers import Input, Lambda,subtract,GlobalMaxPooling2D,Dense,GlobalAveragePooling2D, concatenate, Activation
from keras.applications.mobilenet import MobileNet as Net

from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.layers import Input, merge
from keras.optimizers import Adam
import cv2
import numpy as np
import os
#from tqdm import tqdm
from keras.models import Sequential

from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import time
import glob

ALPHA = 0.7 # Triplet Loss Parameter

from keras.layers import Input,Lambda,subtract,GlobalMaxPooling2D,Dense,GlobalAveragePooling2D,concatenate,Activation
from keras.applications.xception import Xception as Net
from keras.preprocessing import image
from keras.applications.xception import preprocess_input
from keras.models import Model

import matplotlib.pyplot as plt

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix
import random as rn

np.random.seed(0)
tf.set_random_seed(3)

from sklearn.model_selection import cross_val_score

In [ ]:
# run the script containing all framework related functions
%run fn_utils.py

In [ ]:
# load the CNN model
pretrained_model= keras.applications.mobilenet.MobileNet()
layers_needed=pretrained_model.layers[0:82]
#Defining the Sequential model
baseline_model=Sequential()
for layer in layers_needed:
    baseline_model.add(layer)
    
layers_to_not_train= baseline_model.layers[0:79]

for layer in layers_to_not_train:
    layer.trainable=False
    
triplet_model=create_model_new(128, 128, 3, baseline_model)
triplet_model.load_weights('/Users/ochhab3/Documents/ojaswinich_github/signature_verification/model_weights/triplet_model_new_e11.h5')

In [ ]:
#Find the embeddings from the cnn_model
cnn_model = Model(inputs=triplet_model.input,
                                 outputs=triplet_model.get_layer('global_max_pooling2d_1').output)

In [ ]:
# load the logistic model
import pickle
log_model_loaction='/Users/ochhab3/Documents/ojaswinich_github/signature_verification/model_weights/log_model_e11.pkl'
log_model_saved=open(log_model_loaction, 'rb')
log_model=pickle.load(log_model_saved)

In [ ]:
# creating function to input images from folder 1 and folder 2 and save embeddings in database 
def prepare_database_embed(location, cnn_model):
    database = {}

    # load all the images of individuals to recognize into the database
    for file in glob.glob(location):
        identity = os.path.splitext(os.path.basename(file))[0]
        #print (identity)
        identity_array = img_path_to_array(file)
        identity_embed=cnn_model.predict([identity_array,identity_array,identity_array])
        database[identity] = identity_embed

    return database

In [ ]:
# creating base images database dictionary
folder1_location='/Users/ochhab3/Desktop/f1/*' #input path here
folder1_database = prepare_database_embed(folder1_location, cnn_model)

In [ ]:
# creating final test images database dictionary
folder2_location='/Users/ochhab3/Desktop/f2/*' #input path here
folder2_database = prepare_database_embed(folder2_location, cnn_model)

In [ ]:
# empty list for probabilities
final_pred_prob=[['Filename', 'Prob1', 'Prob2', 'Prob3']]

In [ ]:
for test_id in folder2_database:
    person_id= test_id.split('_')[0]
    test_emb=folder2_database[test_id]
    pred_prob_class = [test_id]
    for i in range(1,4):
        anc_emb=folder1_database[person_id+'_'+str(i)]
        diff_emb=anc_emb-test_emb
        pred_prob=log_model.predict_proba(diff_emb)[:,1][0]
        #print (pred_prob)
        pred_prob_class.append(pred_prob)
        #print (pred_prob_class)
    # getting the mode of pred_class
    final_pred_prob.append(pred_prob_class)

In [ ]:
final_pred_prob

In [ ]:
# creating an empty list which will be appended 
final_predictions_any=[['Filename', 'Decision']]

In [ ]:
# proabability threshold to take for forged class
k=0.4

In [ ]:
# if any one is forged, take forged

for test_id in folder2_database:
    person_id= test_id.split('_')[0]
    test_emb=folder2_database[test_id]
    pred_prob_class = []
    for i in range(1,4):
        anc_emb=folder1_database[person_id+'_'+str(i)]
        diff_emb=anc_emb-test_emb
        pred_prob=log_model.predict_proba(diff_emb)[:,1][0]
        #print (pred_prob)
        pred_prob_class.append(pred_prob)
        #print (pred_prob_class)
    # taking final class as forged if any probability >= k, otherwise genuine
    if (pred_prob_class[0] >= k or pred_prob_class[1] >= k or pred_prob_class[2] >= k):
        output='Forgery'
    else:
        output='Original'
    output_entry=[test_id, output]    
    
    final_predictions_any.append(output_entry)

In [ ]:
# get total number of genuine and forged class
x1=np.array(final_predictions_any)[1:]
x2=x1[:,1]
num_orig=len(x2[x2=='Original'])
num_forg=len(x2[x2=='Forgery'])
total=num_orig+num_forg
ratio_forg=num_forg/total
print(num_orig,num_forg,ratio_forg)

In [ ]:
# creating an empty list which will be appended 
final_predictions_mode=[['Filename', 'Decision']]

In [ ]:
# if mode is forged, take forged

for test_id in folder2_database:
    person_id= test_id.split('_')[0]
    test_emb=folder2_database[test_id]
    pred_prob_class = []
    for i in range(1,4):
        anc_emb=folder1_database[person_id+'_'+str(i)]
        diff_emb=anc_emb-test_emb
        pred_prob=log_model.predict_proba(diff_emb)[:,1][0]
        #print (pred_prob)
        pred_prob_class.append(pred_prob)
        #print (pred_prob_class)
    # taking final class as forged if mode of probability >= k, otherwise genuine
    a=sum(i >= k for i in pred_prob_class)
    if (a>=2):
        output='Forgery'
    else:
        output='Original'
    output_entry=[test_id, output]    
    
    final_predictions_mode.append(output_entry)

In [ ]:
# get total number of genuine and forged class
x1=np.array(final_predictions_mode)[1:]
x2=x1[:,1]
num_orig=len(x2[x2=='Original'])
num_forg=len(x2[x2=='Forgery'])
total=num_orig+num_forg
ratio_forg=num_forg/total
print(num_orig,num_forg,ratio_forg)

In [ ]:
# converting the final_predictions_any list to .csv file
import csv
final_predictions_path='final_predictions_any.csv' # give path here
myFile = open(final_predictions_path, 'w')  
with myFile:  
   writer = csv.writer(myFile)
   writer.writerows(final_predictions_any)

In [ ]:
# converting the final_predictions_mode list to .csv file
import csv
final_predictions_path='final_predictions_mode.csv' # give path here
myFile = open(final_predictions_path, 'w')  
with myFile:  
   writer = csv.writer(myFile)
   writer.writerows(final_predictions_mode)

In [ ]:
# converting the final_pred_prob list to .csv file
import csv
final_pred_prob_path='final_probabilities_test.csv' # give path here
myFile = open(final_pred_prob_path, 'w')  
with myFile:  
   writer = csv.writer(myFile)
   writer.writerows(final_pred_prob)